# LLMCheckerChain
This notebook showcases how to use LLMCheckerChain.

In [1]:
from langchain.chains import LLMCheckerChain, LLMChain, SequentialChain, ParallelChain
from langchain.llms import OpenAI
from langchain.prompts.base import ListOutputParser
from langchain.llms.base import LLM


llm = OpenAI(temperature=0.7)

text = "What type of mammal lays the biggest eggs?"

checker_chain = LLMCheckerChain(llm=llm, verbose=True)

# checker_chain.run(text)

In [100]:
# flake8: noqa
from langchain.prompts.prompt import PromptTemplate

_CREATE_DRAFT_ANSWER_TEMPLATE = """Question: {question}

Answer: Let's think step by step. """
CREATE_DRAFT_ANSWER_PROMPT = PromptTemplate(
    input_variables=["question"], template=_CREATE_DRAFT_ANSWER_TEMPLATE
)

_LIST_ASSERTIONS_TEMPLATE = """{statement}
Make a bullet point list of the assumptions you made when producing the above statement.\n\n"""
LIST_ASSERTIONS_PROMPT = PromptTemplate(
    input_variables=["statement"], template=_LIST_ASSERTIONS_TEMPLATE
)

_CHECK_ASSERTIONS_TEMPLATE = """{assertions}
Repeat the assertion above, and then determine if it is true or false. If it is false, explain why.\n\n"""
CHECK_ASSERTIONS_PROMPT = PromptTemplate(
    input_variables=["assertions"], template=_CHECK_ASSERTIONS_TEMPLATE
)

_REVISED_ANSWER_TEMPLATE = """{checked_assertions}

Question: In light of the above assertions and checks, how would you answer the question '{question}'?

The answer is: """
REVISED_ANSWER_PROMPT = PromptTemplate(
    input_variables=["checked_assertions", "question"],
    template=_REVISED_ANSWER_TEMPLATE,
)

In [101]:
llm = OpenAI(temperature=0.7)

create_draft_answer_chain = ParallelChain(
    llm=llm, prompt=CREATE_DRAFT_ANSWER_PROMPT, output_key="statement"
)

list_assertions_chain = ParallelChain(
    llm=llm,
    prompt=LIST_ASSERTIONS_PROMPT,
    output_key="assertions",
    # This indicates that there are multiple outputs
    output_parser=lambda x: [i.strip() for i in x.split("\n") if len(i.strip()) > 0],
)

check_assertions_chain = ParallelChain(
    llm=llm, prompt=CHECK_ASSERTIONS_PROMPT, output_key="checked_assertions"
)

revised_answer_chain = ParallelChain(
    llm=llm,
    prompt=REVISED_ANSWER_PROMPT,
    output_key="revised_answer",
    # This indicates that there are multiple inputs
    input_parser=lambda x: "\n".join(x),
)

self_ask_verification_chain = SequentialChain(
    chains=[
        create_draft_answer_chain,
        list_assertions_chain,
        check_assertions_chain,
        revised_answer_chain,
    ],
    input_variables=["question"],
    output_variables=["revised_answer"],
    verbose=True,
)

In [102]:
# %%time
# self_ask_verification_chain.run(text)

In [124]:
llm = OpenAI(temperature=0.7)

create_draft_answer_chain = ParallelChain(
    llm=llm, prompt=CREATE_DRAFT_ANSWER_PROMPT, output_key="statement", n = 25
)

list_assertions_chain = ParallelChain(
    llm=llm,
    prompt=LIST_ASSERTIONS_PROMPT,
    output_key="assertions",
    # This indicates that there are multiple outputs
#     output_parser=lambda x: [i.strip() for i in x.split("\n") if len(i.strip()) > 0],
)

check_assertions_chain = ParallelChain(
    llm=llm, prompt=CHECK_ASSERTIONS_PROMPT, output_key="checked_assertions"
)

revised_answer_chain = ParallelChain(
    llm=llm,
    prompt=REVISED_ANSWER_PROMPT,
    output_key="revised_answer",
    # This indicates that there are multiple inputs
#     input_parser=lambda x: "\n".join(x),
)


MOE_CONSENSUS_PROMPT = PromptTemplate(
    input_variables=["revised_answer", "question"],
    template="""{revised_answer}

Question: A group of people each gave their answers above. Which answer best answers the question '{question}'?

Answer: """,
)

moe_answer_chain = ParallelChain(
    llm=llm,
    prompt=MOE_CONSENSUS_PROMPT,
    output_key="final_answer",
    # This indicates that there are multiple inputs
    input_parser=lambda x: "\n -" + "\n -".join(x),
)


self_ask_verification_chain = SequentialChain(
    chains=[
        create_draft_answer_chain,
        list_assertions_chain,
        check_assertions_chain,
        revised_answer_chain,
        moe_answer_chain,
    ],
    input_variables=["question"],
    output_variables=["final_answer"],
    verbose=True,
)

In [125]:
%%time
self_ask_verification_chain.run("What type of mammal lays the biggest eggs?")



> Entering new SequentialChain chain...
Chain 0:
{'statement': [' Mammals are animals that produce milk and have fur or hair.  The mammal that lays the biggest eggs is the elephant.  Elephants lay single eggs (ovoids) that weigh up to 11 pounds.', ' Mammals are warm-blooded animals with fur or hair, and that produce milk for their young.  The biggest eggs are laid by birds, not mammals.', ' Mammals lay eggs, but none of them lay really large eggs. The biggest eggs are laid by reptiles, birds, and fish, not mammals. Therefore, the answer is that no mammal lays the biggest eggs.', ' Mammals are warm-blooded, vertebrate animals that typically bear live young.  So, the answer to this question would not be a mammal, as mammals do not lay eggs.  The answer to this question would be a reptile, as reptiles lay the largest eggs of all animals.', "\n\nFirst, let's consider which mammals lay eggs. Mammals that lay eggs include the platypus, echidna, and the spiny anteater.\n\nNext, let's look a

Chain 2:
{'checked_assertions': ['Mammals produce eggs and have fur or hair. False. Mammals do not produce eggs, only birds and reptiles do. Mammals do, however, produce milk and have fur or hair.', 'Assertion: Mammals lay the biggest eggs.\nFalse. Mammals do not lay eggs. Birds lay the biggest eggs, not mammals.', 'The assertion is: No mammal lays the biggest eggs. This statement is true, as mammals do not lay large eggs. Reptiles, birds, and fish lay the biggest eggs.', '\nThe assertion: Mammals lay the largest eggs of all animals\n\nThis assertion is false. Reptiles lay the largest eggs of all animals, not mammals. Mammals typically bear live young, while reptiles lay eggs.', 'Assertion: The ostrich lays the biggest egg of any bird species.\n\nFalse: The ostrich lays the largest egg of any bird species, measuring up to 6 inches long, but the largest egg of any bird species is actually laid by the elephant bird, which can lay eggs up to 12 inches long.', 'The assertion: Elephant lays

[' The echidna, a type of monotreme native to Australia, New Zealand, and other parts of the Pacific region, lays the biggest eggs.']

In [121]:
llm = OpenAI(temperature=0.7)

In [122]:
%%time
question = """Question: What type of mammal lays the biggest eggs?

Answer: Let's think step by step."""
results = llm(question, best_of=25, n = 5)

CPU times: user 9.63 ms, sys: 2.74 ms, total: 12.4 ms
Wall time: 2.37 s


In [123]:
print(sum([int('echidna' in r.lower()) for r in results]))
moe_answer_chain({'revised_answer': results, 'question': question})

0


{'revised_answer': [' What type of animal lays eggs? Birds. What type of bird lays the biggest eggs? The ostrich. Therefore, the type of mammal that lays the biggest eggs is the ostrich.',
  ' All mammals give birth to live young, so none of them lay eggs. Therefore, the correct answer is that no type of mammal lays eggs.',
  ' The biggest eggs are laid by a bird, not a mammal. The largest bird egg is laid by the ostrich, which is a type of bird.',
  ' Mammals are animals that give birth to live young and nurse their young with milk. Since mammals do not lay eggs, the answer to this question is none.',
  " Mammals lay eggs, so the mammal we're looking for is one that lays eggs. The biggest eggs would be laid by the mammal with the largest body size. The mammal that lays the biggest eggs is the elephant."],
 'question': "Question: What type of mammal lays the biggest eggs?\n\nAnswer: Let's think step by step.",
 'final_answer': [' The biggest eggs are laid by a bird, not a mammal. The l

In [9]:
# %%time
# self_ask_verification_chain.run("Where did Sheryl Sandberg meet Mark Zuckerberg?")